# Homework 4 - Recipe Bot Retrieval Evaluation
I am following Option 2 - (Starting with my the provided recipes)

* Load Recipes
* Generate Synthetic Queries
* Build Retriever
* Evaluate Retriever

I have developed a retriever w claude in a module `hybrid_retriever` in the adjacent folder.

# Load Recipes

In [ ]:
from pathlib import Path
from hybrid_retriever.retriever import load_recipes
from hybrid_retriever.models import RecipeBase

processed_recipe_path = Path("reference_files/processed_recipes.json")

recipes = load_recipes(processed_recipe_path)

print(recipes[11].to_embedding_text())

# Generate Synthetic Queries

In [ ]:
from textwrap import dedent
from pydantic import BaseModel

class SyntheticQuery(BaseModel):
    query:str
    salient_fact:str

query_dimensions: list[str] = [
    "finding_recipe",
    "timings",
    "temperatures",
    "methodology",
    "conversions",
    "equipment",
    "ingredient_substitutions",
]

def build_recipe_query(recipe: RecipeBase) -> str:

    query =dedent("""
    # Task
    We are building a RAG chatbot and we want to create some example user queries to evaluate the retriever against.
    Your job is to generate a plausible user query that should result in the provided recipe being retrieved.

    # Instructions
    Your user query must be answered by the content of the source recipe, you must provide evidence of this.
    You must consider the dimension of the query and ensure the user query you provide is on this topic.

    # Example
    Here is an example of the kind of input you will get:
    
    <example_input>
    # Query Topic
    timings

    # Recipe:
    the best vegetarian vegan vegetable samosas

    # Description:
    modified from: “madhur jaffrey’s world-of-the-east vegetarian cooking” these are delicious! the effort of making your own samosa crust, instead of buying phyllo dough, is well worth it. __________________________________________________________ samosas may be served at room temperature or they may be served warm. samosas may be made ahead of time (up to a day), refrigerated neatly in flat plastic containers, and then reheated in a 350 degree oven. if you wish to freeze samosas, fry them partially, drain them, and freeze them in a single layer in flat plastic containers. when you wish to eat them, defrost and fry them a second time.

    # Ingredients:
    - all-purpose white flour
    - salt
    - unsalted butter
    - potatoes
    - vegetable oil
    - onion
    - frozen peas
    - fresh ginger
    - parsley
    - ground coriander
    - garam masala
    - ground roasted cumin seeds
    - lemon juice
    - oil
    - flour

    # Steps:
    1. sift the flour and salt into a bowl
    2. add the softened butter and rub it in with your hands so that the flour resembles fine bread crumbs
    3. add warm water , a tablespoon at a time , and begin to gather the flour into a ball
    4. you will need 5 tablespoons of water
    5. form a ball and begin to knead it
    6. knead well for about 10-15 minutes or until dough is very soft and pliable
    7. (if you have a food processor , put the steel blade in place and empty the sifted flour and salt into a container
    8. add the softened butter and turn on the machine
    9. when you have a bread-crumb consistency , begin to add about 5 tablespoons of water slowly through the funnel
    10. stop when the dough forms a ball
    11. take out the ball and knead it for 5-10 minutes or until it is very soft and pliable
    12. )
    13. wrap the dough in plastic wrap and let it sit for an hour in the refrigerator
    14. the dough can be made a day in advance and refrigerated
    15. make the stuffing
    16. peel the potatoes and dice them into roughly 1 / 4-inch pieces
    17. heat the 4 tablespoons oil in a 10-12 skillet over a medium flame
    18. put in the onion , stirring and frying until it turns a light-brown color
    19. add the peas , the ginger , chinese parsley , and 3 tablespoons of water
    20. cover , lower heat and simmer very gently until peas are cooked
    21. stir every now and then and add additional water , a tablespoon at a time , if the skillet seems dried out
    22. now put in the diced potatoes , salt , coriander , garam masala , roasted ground cumin , & lemon juice
    23. keep heat on low and mix the spices with the potatoes
    24. continue cooking gently , stirring frequently , for 3-4 minutes
    25. check salt and lemon juice
    26. turn off heat and leave potato mixture to cool
    27. take the dough out of the refrigerator and knead again
    28. divide dough into 12 equal balls
    29. keep balls covered with plastic wrap
    30. place a small bowl of water on your work surface
    31. lightly flour on a pastry board
    32. flatten one of the dough balls on it and roll it out into a round about 6 in diameter
    33. now cut the round in half with a sharp knife
    34. pick up one half and form a cone , making a 1 / 4 overlapping seam
    35. using a little water , from the nearby bowl to create the seam
    36. fill the cone with a heaping tablespoon of the stuffing
    37. close the top of the cone by sticking the open edges of the triangle together , again with the help of a little water
    38. this seam should also be 1 / 4 wide
    39. press the top seam again and , if possible , flute it with your fingers
    40. put the samosa on a platter in a cool spot
    41. make all 24 samosas this way
    42. heat oil for deep frying in a wok or other wide utensil over medium-low flame
    43. when the oil is hot , drop in the samosas , as many as will lie in a single layer
    44. fry them slowly until they are golden brown , turning them over when one side seems done
    45. when the second side of the samosas has turned a golden color , remove them from the oil with a slotted spoon and place them on a paper-towel-lined platter
    46. do all samosas this way

    # Tags: weeknight, time-to-make, course, cuisine, preparation, occasion, for-1-or-2, appetizers, side-dishes, asian, indian, dinner-party, holiday-event, vegan, vegetarian, dietary, inexpensive, superbowl, number-of-servings, 4-hours-or-less, leftovers

    # Metadata
    Cooking Time: 80 minutes | Number of Ingredients: 15 | Number of Steps: 46
    </example_input>

    Here is an example of the kind of output we are looking for:
    <example_output>If i'm making samosas, how long do I need to knead the dough for?</example_output>

    # Input
    Ok - now you:
    <input>
    # Query Topic
    {query_dimension}
    
    {recipe}
    </input>

    # Formatting
    Return your output as json in the format:
    {'query':'If i'm making samosas...','salient_fact':'step 11 says: take out the ball and knead it for 5-10 minutes or until it is very soft and pliable'}
    """)

    return query